Transformer 구현

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

In [ ]:
# Positional Encoding (위치 인코딩)
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)  # 위치 인코딩을 저장할 행렬 초기화
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # 위치값 생성
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # 짝수 인덱스에 사인 함수 적용
        pe[:, 1::2] = torch.cos(position * div_term)  # 홀수 인덱스에 코사인 함수 적용
        pe = pe.unsqueeze(0)  # 배치 차원 추가
        self.register_buffer('pe', pe)  # 학습되지 않는 버퍼로 등록
    
    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]  # 입력 텐서에 위치 인코딩 추가

In [ ]:
# Multi-Head Self Attention (다중 헤드 자기 어텐션)
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0  # d_model이 num_heads로 나누어 떨어져야 함
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads  # 각 헤드의 차원 크기
        
        # Q, K, V를 위한 선형 변환 레이어
        self.qkv_linear = nn.Linear(d_model, d_model * 3)
        self.out_linear = nn.Linear(d_model, d_model)  # 최종 출력 변환
        self.softmax = nn.Softmax(dim=-1)  # 어텐션 가중치 계산

In [ ]:
def forward(self, x):
        batch_size, seq_length, d_model = x.shape
        
        # Q, K, V로 변환 및 다중 헤드 분할
        qkv = self.qkv_linear(x).view(batch_size, seq_length, 3, self.num_heads, self.d_k)
        q, k, v = qkv.permute(2, 0, 3, 1, 4)  # (3, batch_size, num_heads, seq_length, d_k)
        
        # 어텐션 점수 계산 및 스케일링
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn = self.softmax(scores)  # 어텐션 가중치
        
        # 어텐션 가중치를 V에 적용하여 컨텍스트 벡터 생성
        context = torch.matmul(attn, v).permute(0, 2, 1, 3).contiguous().view(batch_size, seq_length, d_model)
        return self.out_linear(context)  # 최종 출력 변환

In [ ]:
# Feed Forward Network (포지션별 피드 포워드 네트워크)
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionwiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)  # 첫 번째 완전 연결층
        self.fc2 = nn.Linear(d_ff, d_model)  # 두 번째 완전 연결층
        self.relu = nn.ReLU()
    
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))  # 활성화 함수 적용 후 변환

In [ ]:
# Transformer Encoder Layer (트랜스포머 인코더 레이어)
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super(TransformerEncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)  # 다중 헤드 어텐션
        self.norm1 = nn.LayerNorm(d_model)  # 정규화 레이어
        self.ffn = PositionwiseFeedForward(d_model, d_ff)  # 피드 포워드 네트워크
        self.norm2 = nn.LayerNorm(d_model)  # 정규화 레이어
    
    def forward(self, x):
        attn_output = self.attention(x)  # 자기 어텐션 수행
        x = self.norm1(x + attn_output)  # 잔차 연결 및 정규화
        ffn_output = self.ffn(x)  # 피드 포워드 네트워크 수행
        return self.norm2(x + ffn_output)  # 잔차 연결 및 정규화

In [ ]:
# Transformer Model (트랜스포머 모델)
class Transformer(nn.Module):
    def __init__(self, d_model=512, num_heads=8, d_ff=2048, num_layers=6, vocab_size=10000):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)  # 단어 임베딩
        self.positional_encoding = PositionalEncoding(d_model)  # 위치 인코딩
        self.encoder = TransformerEncoder(d_model, num_heads, d_ff, num_layers)  # 인코더
        self.fc_out = nn.Linear(d_model, vocab_size)  # 최종 출력 레이어
    
    def forward(self, x):
        x = self.embedding(x)  # 단어를 벡터로 변환
        x = self.positional_encoding(x)  # 위치 정보 추가
        x = self.encoder(x)  # 인코더에 입력
        return self.fc_out(x)  # 최종 출력

In [ ]:
# 모델 생성 및 테스트
model = Transformer()
test_input = torch.randint(0, 10000, (1, 10))  # 랜덤한 10개 토큰 시퀀스
test_output = model(test_input)
print(test_output.shape)  # 예상 출력: (1, 10, 10000) (각 단어에 대한 확률 분포)